In [ ]:
import sys
import glob
import numpy as np
import pandas as pd
from path import Path
from Bio import Seq, SeqIO, AlignIO, Phylo, Align
sys.path.append('../src/')

In [ ]:
sample_sheet = pd.read_csv('210406_SARS-CoV-2_samplesheet.csv', skiprows=19)
sample_sheet

In [ ]:
sample_sheet['Sample_ID'].unique().shape

In [ ]:
def separate_samples(sample_sheet: pd.DataFrame, 
                     private_ids: list=['LASV'], 
                     primer_ids: list=['iTru']) -> dict:
    """RELEASE functionality: Separate A-lab Sample Sheet into constituent sub-sheets for each primer set."""
    results = {}
    samples_to_exclude = []
    # separate out private sample IDs
    results['_private_sheet'] = pd.concat([sample_sheet.loc[sample_sheet['Sample_ID'].str.contains(idx)] for idx in private_ids])
    for sample_idx in results['_private_sheet']['Sample_ID'].unique():
            samples_to_exclude.append(sample_idx)
    for primer_idx in primer_ids:
        results[f'{primer_idx}_primers'] = sample_sheet.loc[sample_sheet['I7_Index_ID'].str.contains(primer_idx)]
        for sample_idx in results[f'{primer_idx}_primers']['Sample_ID'].unique():
            samples_to_exclude.append(sample_idx)
    results['OG_primers'] = sample_sheet.loc[~sample_sheet['Sample_ID'].isin(samples_to_exclude)]
    return results
res = separate_samples(sample_sheet)

In [ ]:
res['_private_sheet'].to_csv('_SampleSheet.csv', index=False)

res['iTru_primers'].to_csv('itru_primer_SampleSheet.csv', index=False)

res['OG_primers'].to_csv('old_primer_SampleSheet.csv', index=False)

In [ ]:
lins = pd.read_csv('/Users/al/Documents/scripps/analysis/alab_analysis/2021.03.29/2021-03-29_analysis_report_v2/old_primers_batch/msa/lineage_report_2021-03-30.csv')
lins

In [ ]:
p1_sample_ids = lins.loc[lins['lineage']=='P.1', 'taxon'].apply(lambda x: x.split('_')[1]).tolist()

In [ ]:
gisaid_meta = pd.read_csv('/Users/al/Documents/scripps/analysis/alab_release/2021-03-31_release/valhalla/2021-03-31_release/gisaid_metadata.csv')
gisaid_meta

In [ ]:
gisaid_meta['sample_id'] = gisaid_meta['Virus name'].apply(lambda x: x.split('/')[2])
gisaid_meta['sample_id']

In [ ]:
gisaid_meta.loc[gisaid_meta['sample_id'].isin(p1_sample_ids), 'Additional location information'] = 'travel-related'

In [ ]:
gisaid_meta.drop(columns=['sample_id'], inplace=True)

In [ ]:
gisaid_meta.to_csv('/Users/al/Documents/scripps/analysis/alab_release/2021-03-31_release/valhalla/2021-03-31_release/gisaid_metadata.csv', index=False)

In [ ]:
git_meta = pd.read_csv('/Users/al/Documents/scripps/analysis/alab_release/2021-03-31_release/valhalla/2021-03-31_release/metadata.csv')
git_meta['collection_date'] = git_meta['collection_date'].astype(str)
git_meta.to_csv('/Users/al/Documents/scripps/analysis/alab_release/2021-03-31_release/valhalla/2021-03-31_release/metadata.csv', index=False)

In [ ]:
d = {'a': 2}
list(d.values())

In [ ]:
%load_ext autoreload
%autoreload 2
from alab_release import *
import bjorn_support as bs
import mutations as bm
# import unsupervised_learning as bul

In [ ]:
%reload_ext autoreload

In [ ]:
s_ids = [f"SEARCH-{idx}" for idx in [7338, 7443, 7458, 7600]]

In [ ]:
s = pd.read_csv('/Users/al/Documents/scripps/analysis/bjorn/test_analysis/SARS-CoV-2_sequence_tracker-GISAID.csv')
s = s.loc[s['SEARCH SampleID'].isin(s_ids)]

In [ ]:
s.to_csv('/Users/al/Documents/scripps/analysis/bjorn/test_analysis/SARS-CoV-2_sequence_tracker-GISAID.csv', index=False)

In [ ]:
# subs_fp = '/valhalla/dev/replacements.csv'
# subs = pd.read_csv(subs_fp)
# subs

In [ ]:
msa = bs.load_fasta('/valhalla/dev/msa/dev_aligned.fa', is_aligned=True)
meta_fp = '/valhalla/dev/metadata.csv'
inss = bm.identify_insertions(msa, meta_fp)
subs = bm.identify_replacements(msa, meta_fp)
dels = bm.identify_deletions(msa, meta_fp)

In [ ]:
subs['gene'].unique()

In [ ]:
nonconcerning_genes = ['5UTR', 'ORF7a', 'ORF7b', 'ORF8', 'ORF10', 'Non-coding region']
subs_flag = ((subs['alt_aa']=='*') & (~subs['gene'].isin(nonconcerning_genes)))
sus_subs_ids = subs.loc[subs_flag, 'samples'].str.split(',').explode().unique().tolist()
sus_subs_ids

In [ ]:
subs.loc[subs_flag]

In [ ]:
dels_flag = ((dels['is_frameshift']==True) & (~dels['gene'].isin(nonconcerning_genes)))
sus_dels_ids = dels.loc[dels_flag, 'samples'].str.split(',').explode().unique().tolist()
sus_dels_ids

In [ ]:
dels.loc[dels_flag]

In [ ]:
inss

In [ ]:
inss[(inss['is_frameshift']==True)]

In [ ]:
inss.loc[ins_flag, 'samples']

In [ ]:
ins_flag = ((inss['is_frameshift']==True) & (~inss['gene'].isin(nonconcerning_genes)))
sus_ins_ids = inss.loc[ins_flag, 'samples'].str.split(',').explode().unique().tolist()
sus_ins_ids

In [ ]:
sus_ids = list(set(sus_subs_ids + sus_dels_ids + sus_ins_ids))

In [ ]:
sus_ids

In [ ]:
msa = bs.load_fasta('/valhalla/dev/msa/dev_aligned.fa', is_aligned=True)
good_seqs = []
poor_seqs = []
for rec in msa:
    if rec.id in sus_ids:
        poor_seqs.append(rec)
    else:
        good_seqs.append(rec)

In [ ]:
good_msa = Align.MultipleSeqAlignment(good_seqs)
poor_msa = Align.MultipleSeqAlignment(poor_seqs)


In [ ]:
good_msa

In [ ]:
poor_msa

In [ ]:
AlignIO.write(poor_msa, 'test.fa', 'fasta')

In [ ]:
s_fp = '/valhalla/fastq/2021.03.19/samples.tsv'
s_df = pd.read_csv(s_fp, sep='\t')
s_df

In [ ]:
m_fp = '/home/al/code/HCoV-19-Genomics/metadata.csv'
m = pd.read_csv(m_fp)
m.tail(15)

In [ ]:
m.sort_values('collection_date', ascending=False).head(20)

In [ ]:
samples_fp = '/valhalla/fastq/2021.03.19/Fastq/samples.tsv'
samples = pd.read_csv(samples_fp, sep='\t')
samples

In [ ]:
samples['sample'].unique().shape

In [ ]:
fa_fp = '/valhalla/dev/msa/dev.fa'
ref_fp = '/home/al/data/hcov19/NC045512.fasta'
viral_msa_fp = '/home/al/code/ViralMSA.py'
bs.align_fasta_viralMSA(fa_fp, '/valhalla/dev/msa/test_msa', ref_fp=ref_fp, viralmsa_fp=viral_msa_fp)

In [ ]:
msa_data = bs.load_fasta('/valhalla/dev/msa/test_msa/dev.fa.aln', is_aligned=True)
list(msa_data)

In [ ]:
subs_fp = '/valhalla/2021-03-17_release/replacements.csv'
subs = pd.read_csv(subs_fp)
subs.loc[subs['alt_aa']=='*']

In [ ]:
inserts_fp = '/valhalla/dev/insertions.csv'
inserts = pd.read_csv(inserts_fp)
inserts

In [ ]:
inserts_fp = '/valhalla/dev/deletions.csv'
inserts = pd.read_csv(inserts_fp)
inserts

In [ ]:
meta_fp = '/home/al/code/HCoV-19-Genomics/metadata.csv'
meta_df = pd.read_csv(meta_fp)
meta_df

In [ ]:
# meta_df.to_csv('/home/al/code/HCoV-19-Genomics/metadata.csv', index=False)

In [ ]:
3827-3575

In [ ]:
# tijuana
283-275

In [ ]:
# imperial
131-1

In [ ]:
# riverside
28 -23

In [ ]:
# meta_df.loc[meta_df['location']=='MEX/Baja California/Tijuana', 'location'] = 'Mexico/Baja California/Tijuana'

In [ ]:
meta_df['location'].value_counts().sort_index(ascending=True)

In [ ]:
analysis_fpath = '/valhalla/analysis'
sample_sheet_fpath = '/home/al/analysis/alab_release/SARS-CoV-2_sequence_tracker-GISAID.csv'
released_samples_fpath = '/home/al/code/HCoV-19-Genomics/metadata.csv'
include_bams = True

In [ ]:
# grab all filepaths for bam data
bam_filepaths = glob.glob(f"{analysis_fpath}/**/merged_aligned_bams/illumina/*.bam")
bam_filepaths = [Path(fp) for fp in bam_filepaths]
# consolidate sample ID format
bam_ids = get_ids(bam_filepaths)
# Turn into dataframe
bam_data = list(zip(*[bam_ids, bam_filepaths]))
bam_df = pd.DataFrame(data=bam_data, columns=['sample_id', 'PATH'])
# grab all paths to consensus sequences
consensus_filepaths = glob.glob(f"{analysis_fpath}/**/consensus_sequences/illumina/*.fa")
consensus_filepaths = [Path(fp) for fp in consensus_filepaths]
# consolidate sample ID format
consensus_ids = get_ids(consensus_filepaths)
# Turn into dataframe
consensus_data = list(zip(*[consensus_ids, consensus_filepaths]))
consensus_df = pd.DataFrame(data=consensus_data, columns=['sample_id', 'PATH'])
# clean up cns and bam (remove duplicate IDs)
bam_df.drop_duplicates(subset=['sample_id'], keep='last', inplace=True)
consensus_df.drop_duplicates(subset=['sample_id'], keep='last', inplace=True)
# include only SEARCH samples
consensus_df = consensus_df[(consensus_df['sample_id'].str.contains('SEARCH'))]
# merge consensus and bam filepaths for each sample ID
analysis_df = pd.merge(consensus_df, bam_df, on='sample_id', how='left')
# load sample sheet data (GISAID) - make sure to download most recent one
seqsum = pd.read_csv(sample_sheet_fpath)
# clean up
seqsum = seqsum[(~seqsum['SEARCH SampleID'].isna()) & (seqsum['SEARCH SampleID']!='#REF!')]
# consolidate sample ID format
seqsum.loc[:, 'sample_id'] = seqsum['SEARCH SampleID'].apply(process_id)
seqsum.drop_duplicates(subset=['sample_id'], keep='last', inplace=True)
seqsum = seqsum[seqsum['New sequences ready for release'] == 'Yes']
num_seqs_to_release = seqsum['sample_id'].unique().shape[0]
# JOIN summary sheet with analysis meta data
sequence_results = pd.merge(seqsum, analysis_df, on='sample_id', how='inner')
# compute number of samples with missing consensus and/or bam files
num_seqs_found = sequence_results['sample_id'].unique().shape[0]
num_samples_missing_cons = num_seqs_to_release - num_seqs_found
num_samples_missing_bams = 'NA'
if include_bams:
    # exclude any samples that do not have BAM data
    num_samples_missing_bams = sequence_results[sequence_results['PATH_y'].isna()].shape[0]
    sequence_results = sequence_results[~sequence_results['PATH_y'].isna()]
# samples missing consensus or BAM sequence files
num_samples_missing_bams = sequence_results[sequence_results['PATH_y'].isna()].shape[0]
num_samples_missing_cons = sequence_results[sequence_results['PATH_x'].isna()].shape[0]
# ## Make sure to remove any samples that have already been uploaded to github (just an extra safety step)
# load metadata.csv from github repo, then clean up
meta_df = pd.read_csv(released_samples_fpath)
meta_df = meta_df[meta_df['ID'].str.contains('SEARCH')]
# consolidate sample ID format
meta_df.loc[:, 'sample_id'] = meta_df['ID'].apply(process_id)
# meta_df['sample_id']
# get IDs of samples that have already been released
released_seqs = meta_df['sample_id'].unique()
# filter out released samples from all the samples we got
final_result = sequence_results[~sequence_results['sample_id'].isin(released_seqs)]
print(f"Preparing {final_result.shape[0]} samples for release")
# ## Getting coverage information
cov_filepaths = glob.glob("{}/**/trimmed_bams/illumina/reports/*.tsv".format(analysis_fpath))
# get_ipython().getoutput("find {analysis_fpath} -type f -path '*trimmed_bams/illumina/reports*' -name '*.tsv'")
cov_filepaths = [Path(fp) for fp in cov_filepaths]
# read coverage data and clean it up
cov_df = pd.concat((pd.read_csv(f, sep='\t').assign(path=f) for f in cov_filepaths))
cov_df.loc[:,'sample_id'] = cov_df['SAMPLE'].apply(process_coverage_sample_ids)
cov_df.loc[:,'date'] = cov_df['path'].apply(lambda x: ''.join(x.split('/')[4].split('.')[:3]))
cov_df = (cov_df.sort_values('date')
          .drop_duplicates(subset=['sample_id'], keep='last'))
# JOIN results with coverage info
ans = (
pd.merge(final_result, cov_df,
         on='sample_id', how='left')
  .assign(
    collection_date = lambda x: pd.to_datetime(x["Collection date"]).dt.strftime("%Y-%m-%d")
)
  .rename(columns={
    "SEARCH SampleID": "ID",
    "Location": "location",
    "COVERAGE": "percent_coverage_cds",
    "AVG_DEPTH": "avg_depth",
    "Authors": "authors",
    "Originating lab": "originating_lab"
})
)
ans['fasta_hdr'] = ans['Virus name']
num_samples_missing_coverage = ans[ans['percent_coverage_cds'].isna()].shape[0]
# compute number of samples below 90% coverage
low_coverage_samples = ans[ans["percent_coverage_cds"] < 90]
# ignore samples below 90% coverage
ans = ans[ans["percent_coverage_cds"] >= 90]

In [ ]:
seqsum

In [ ]:
seqsum = pd.read_csv(sample_sheet_fpath)
seqsum['New sequences ready for release'].unique()

In [ ]:
fp = '/valhalla/fastq/2021.03.12/samples.tsv'
samples = pd.read_csv(fp, sep='\t')
samples

In [ ]:
api_data_fp = 'new_api_data.json.gz'
gcloud_stat_cmd = f'/home/al/code/google-cloud-sdk/bin/gsutil stat gs://andersen-lab_temp/outbreak_genomics/new_api_data.json.gz'
t = bs.run_command_log(gcloud_stat_cmd)
t

In [ ]:
t

In [ ]:
fp = '/valhalla/gisaid/workflow/mutations_2021-03-09.csv'
muts = pd.read_csv(fp)

In [ ]:
muts.loc[muts['country']=='United States', 'location_id']

In [ ]:
muts = muts[muts['is_synonymous']==False]

In [ ]:
lineage = 'B.1.526'
data = (muts.loc[muts['pangolin_lineage']==lineage, 'mutation']
        .value_counts()
        .to_frame()
        .reset_index()
        .rename(columns={'index': 'name', 'mutation': 'count'}))#.values
data['frequency'] = (data['count'] - np.min(data['count']))/np.ptp(data['count'])
data = data[data['frequency']>=0.05]
# data

In [ ]:
from scipy.spatial import distance
from scipy.stats import mode
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import SpectralClustering
from sklearn.metrics import confusion_matrix, accuracy_score
from skimage.io import imread

In [ ]:
def kmeans_unsupervised(X:np.ndarray, k:int, centroids=None, tolerance=1e-2):
    """K-Means Clustering with unsupervised learning trick to infer characteristic mutations
    for a given SARS-CoV-2 lineage"""
    # Initialize Input, Centroids, Clusters
    if len(X.shape) < 2: X = np.expand_dims(X, axis=1)
    if centroids: 
        centroids = bul.init_centroids(X, k)
    else:
        centroids = X[np.random.choice(X.shape[0], size=k, replace=False), :]
    clusters = [[] for c in centroids]
    # Until Convergence
    while(True):
        # Centroid Norm at t-1
        old_centroids_norm = np.linalg.norm(centroids)
        # Minimum Distance to Centroid Per Record
        js = np.argmin(distance.cdist(X, centroids), axis=1)
        # Assign Records to Clusters
        for i, c in enumerate(clusters):
             clusters[i] = list(np.where(js == i)[0])
        # Recompute Cluster Centroids
        for j, c in enumerate(centroids):
            centroids[j] = np.mean(X[clusters[j]], axis=0)
        # Centroid Norm at t
        centroids_norm = np.linalg.norm(centroids)
        # Convergence!
        if np.abs(centroids_norm - old_centroids_norm) < tolerance:
            preds = np.zeros_like(X)
            clusters = np.asarray(clusters)[np.argsort(centroids.squeeze())[::-1]]
#             return clusters, centroids
            for i, c in enumerate(clusters):
                preds[c] = i
            return preds.squeeze()

In [ ]:
X = np.expand_dims(data['frequency'].to_numpy(), axis=1)
preds = kmeans_unsupervised(X, k=2, centroids="kmeans++") == 0
preds

In [ ]:
# data['is_characteristic'] = bul.spectral_clustering_unsupervised(X, num_classes=2, n_trees=100)

In [ ]:
data['is_characteristic'] = preds
data['lineage'] = lineage
data.to_csv(f'/Users/al/Documents/scripps/analysis/bjorn/{lineage}_char_mutations.csv', index=False)

In [ ]:
data